## Scrublet

In [ ]:
# install all necessary libraries
pip install scrublet

In [ ]:
# import those libraries
import scanpy as sc
import pandas as pd
import seaborn as sns
%matplotlib inline
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os

### AnnData

In [ ]:
# Importing scRNA-seq h5ad file
adata = sc.read_h5ad("/data/dataset_before_doublet_elimination.h5ad")

In [ ]:
# checking the metadata layers from anndata
adata

In [ ]:
# retrieve umap with patient information on metadata
sc.pl.umap(adata, color=['patient'], frameon=True, 
           ncols=1, palette=pal, size=5,
           legend_fontsize='xx-small', legend_fontweight= 'light',
           title='')

In [ ]:
# other way to check de layers of information on the metadata of the anndata file
metadata_df.head(20)

In [ ]:
# check the amount of patients present in the dataset
adata.obs['patient'].value_counts()

### Subset

In [ ]:
# subset annData per patient
adata1 = adata[adata.obs['patient'].isin(['XXXX'])] # repete that line as many time as the number of patients in the dataset. Change the name of the variable for each patient.

In [ ]:
# check the amount of cells for each patient
print(adata1.obs['patient'].value_counts())

In [ ]:
# calculate the quality control metrics using scanpy for all patients in the original dataset
sc.pp.calculate_qc_metrics(adata, expr_type='counts', var_type='genes', qc_vars=(), 
                           percent_top=None, layer=None, use_raw=False, inplace=True, 
                           log1p=False, parallel=None)

In [ ]:
# generate plots for all the quality control metrics
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, groupby = 'patient', rotation= 45)

### Remove doublet

In [ ]:
# repete the next 5 steps for each patient subset of the dataset
# this step run the scrublet with automated threshold
scrub = scr.Scrublet(adata1.X)
adata1.obs['doublet_scores'], adata1.obs['predicted_doublets'] = scrub.scrub_doublets()
scrub.plot_histogram()
print(sum(adata1.obs['predicted_doublets']) , 'doublet(s) detected')

print('Running UMAP...')
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
print('Done.')

scrub.plot_embedding('UMAP', order_points=True);

In [ ]:
# in that step run scrublet changing the threshold value (if necessary) by looking the histograms in the previous step
scrub = scr.Scrublet(adata1.X)
adata1.obs['doublet_scores'], adata1.obs['predicted_doublets'] = scrub.scrub_doublets()
adata1.obs['predicted_doublets'] = scrub.call_doublets(threshold=0.38)
adata1.obs['doublet_info'] = adata1.obs["predicted_doublets"].astype(str)
scrub.plot_histogram()
print(sum(adata1.obs['predicted_doublets']) , 'doublet(s) detected')

print('Running UMAP...')
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
print('Done.')

scrub.plot_embedding('UMAP', order_points=True);

sc.pp.neighbors(adata1, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata1)
sc.pl.umap(adata1, color=['ratio_nCount_nFeature', 'doublet_info'])

In [ ]:
# recalculate the quality control metrics from scanpy
sc.pp.calculate_qc_metrics(adata1, expr_type='counts', var_type='genes', qc_vars=(), percent_top=None, layer=None, use_raw=False, inplace=True, log1p=False, parallel=None)

In [ ]:
# rerun the violin plot using the new calculated quality control metrics
sc.pl.violin(adata1, 'n_genes_by_counts',
             jitter=0.4, groupby = 'doublet_info', rotation=45)

In [ ]:
# concatenate all subsets of patients in one final dataset
adata_def = adata1.concatenate(adata2, adata3)

In [ ]:
# recalculate the quality control metrics for all patients together in order to compare with the plots in the beggining 
sc.pp.calculate_qc_metrics(adata_def, expr_type='counts', var_type='genes', qc_vars=(), percent_top=None, layer=None, use_raw=False, inplace=True, log1p=False, parallel=None)

In [ ]:
# plot the final violin plots of the amount of true doublets identified in all dataset
sc.pl.violin(adata_def, 'n_genes_by_counts',
             jitter=0.4, groupby = 'doublet_info', rotation=45)

In [ ]:
# this will eliminate the doublet from the final dataset
adatadef = adatadef[adata_def.obs['doublet_info'].isin(['False'])]

In [ ]:
# final violin plot to compare quality control metrics after doublet elimination
sc.pl.violin(adata_def, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, groupby = 'patient', rotation= 45)

In [ ]:
# check the final amount of cells remained in the dataset after doublet detection
adata_def

In [ ]:
# save the final h5ad file of the dataset after doublet elimination
results_file = '/data/results/'  # the file that will store the analysis results
author = 'dataset_after_doublet_elimination.h5ad'
adata_def.write(results_file + author, compression='gzip')